In [11]:
import pandas as pd
from glob import glob

In [12]:
scenario_id = "3-1"
scenario_path = "/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware-Capture-"+scenario_id+"/"

In [13]:
!cd {scenario_path}bro && unexpand -t 3 conn.log.labeled > conn.log && sed -i '1,8d' conn.log && sed 's/ //g' -i conn.log && <<< $(cat ../../../base.log) < conn.log | sponge conn.log
!cd {scenario_path}zeek && sed -i '1,8d' flowmeter.log && sed 's/ //g' -i flowmeter.log && <<< $(cat ../../../base_zeek.log) < flowmeter.log | sponge flowmeter.log

In [14]:
conn = pd.read_csv(scenario_path + 'bro/conn.log', sep="\t", header=0, low_memory=False)
conn.drop(conn.shape[0]-1, inplace=True)
conn = conn[["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state", "label", "detailed-label"]]
conn.columns

Index(['ts', 'uid', 'id.orig_h', 'id.resp_h', 'id.resp_p', 'proto', 'service',
       'conn_state', 'label', 'detailed-label'],
      dtype='object')

In [15]:
flow = pd.read_csv(scenario_path + 'zeek/flowmeter.log', sep="\t", header=0, low_memory=False, index_col=False)
flow.drop(flow.shape[0]-1, inplace=True)
flow.drop("uid", axis=1, inplace=True)
flow.columns

Index(['flow_duration', 'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
       'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
       'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot',
       'fwd_header_size_min', 'fwd_header_size_max', 'bwd_header_size_tot',
       'bwd_header_size_min', 'bwd_header_size_max', 'flow_FIN_flag_count',
       'flow_SYN_flag_count', 'flow_RST_flag_count', 'fwd_PSH_flag_count',
       'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_URG_flag_count',
       'bwd_URG_flag_count', 'flow_CWR_flag_count', 'flow_ECE_flag_count',
       'fwd_pkts_payload.min', 'fwd_pkts_payload.max', 'fwd_pkts_payload.tot',
       'fwd_pkts_payload.avg', 'fwd_pkts_payload.std', 'bwd_pkts_payload.min',
       'bwd_pkts_payload.max', 'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg',
       'bwd_pkts_payload.std', 'flow_pkts_payload.min',
       'flow_pkts_payload.max', 'flow_pkts_payload.tot',
       'flow_pkts_payload.avg', 'flow_pkts_payload.std', 'fwd

In [16]:
dataset = pd.concat([conn, flow], axis=1)
dataset.drop(["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state"], axis=1, inplace=True)
dataset.head(1)

label             detailed-label flow_duration  fwd_pkts_tot  \
0  Malicious  PartOfAHorizontalPortScan      2.998333           3.0   

   bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  \
0           0.0                0.0                0.0          1.000556   

   bwd_pkts_per_sec  flow_pkts_per_sec  ...  active.std  idle.min  idle.max  \
0               0.0           1.000556  ...         0.0       0.0       0.0   

   idle.tot  idle.avg  idle.std  fwd_init_window_size  bwd_init_window_size  \
0       0.0       0.0       0.0               29200.0                   0.0   

   fwd_last_window_size  bwd_last_window_size  
0               29200.0                   0.0  

[1 rows x 82 columns]

In [17]:
benign = dataset[dataset["label"] == "Benign"]
benign.to_csv('/media/kmdr7/Seagate/DATASETS/Merged-IoT-Dataset/Benign/'+scenario_id+'.csv', index=False)
benign.shape

(4536, 82)

In [18]:
malware = dataset[dataset["label"] != "Benign"]
malware.to_csv('/media/kmdr7/Seagate/DATASETS/Merged-IoT-Dataset/Malware/'+scenario_id+'.csv', index=False)
malware.shape

(151567, 82)

In [22]:
malware.isnull().any().any()

False